In [7]:
from datetime import datetime
import json
import numpy as np
import os
import random

import tensorflow as tf

import data_loader
import losses
import model
import cv2

from stats_func import *

In [14]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
source_train_pth = './road/source_train.txt'
target_train_pth = './road/target_train_'
source_val_pth = './road/source_val.txt'
target_val_pth = './road/target_val_'


evaluation_interval = 10
save_interval = 300
num_cls = 4
keep_rate_value=0.75
is_training_value=True

BATCH_SIZE = 4

class PCH:

    def __init__(self, pool_size, lambda_a,
                 lambda_b, output_root_dir, to_restore,
                 base_lr, max_step,
                 checkpoint_dir, do_flipping, skip, kfold):
        current_time = datetime.now().strftime("%Y%m%d-%H%M%S")

        self._source_train_pth = source_train_pth
        self._target_train_pth = target_train_pth
        self._source_val_pth = source_val_pth
        self._target_val_pth = target_val_pth
        self._num_cls = num_cls
        tf.compat.v1.disable_eager_execution()
        self.keep_rate = tf.compat.v1.placeholder(tf.float32, shape=())
        self.is_training = tf.compat.v1.placeholder(tf.bool, shape=())
        self._pool_size = pool_size
        self._size_before_crop = 256
        self._lambda_a = lambda_a
        self._lambda_b = lambda_b
        self._output_dir = os.path.join(output_root_dir, str(kfold))
        self._images_dir = os.path.join(self._output_dir, 'imgs')
        self._num_imgs_to_save = 20
        self._to_restore = to_restore
        self._base_lr = base_lr
        self._max_step = max_step
        self._checkpoint_dir = checkpoint_dir
        self._do_flipping = do_flipping
        self._skip = skip
        self._kfold = kfold

        self.fake_images_A = np.zeros(
            (self._pool_size, BATCH_SIZE, model.IMG_HEIGHT, model.IMG_WIDTH, 1)
        )
        self.fake_images_B = np.zeros(
            (self._pool_size, BATCH_SIZE, model.IMG_HEIGHT, model.IMG_WIDTH, 1)
        )


    def train(self):
        # 模型训练

        # 从数据集文件夹加载数据集
        TTrain = self._target_train_pth + str(self._kfold) +'.txt'     # 训练源域图片名称
        TVal = self._target_val_pth + str(self._kfold) +'.txt'        # 训练时目标域图片名称

        self.inputs = data_loader.load_data(self._source_train_pth, TTrain, True)   # 调用data_load文件
        self.inputs_val = data_loader.load_data(self._source_val_pth, TVal, True)   # 验证集文件（源域，目标域）（均为txt）

        

def main(log_dir, config_filename, checkpoint_dir, skip, kfold):

    if not os.path.isdir(log_dir):
        os.makedirs(log_dir)

    with open(config_filename) as config_file:
        config = json.load(config_file)

    tf.compat.v1.reset_default_graph()

    # 定义超参数
    lambda_a = float(config['_LAMBDA_A']) if '_LAMBDA_A' in config else 10.0
    lambda_b = float(config['_LAMBDA_B']) if '_LAMBDA_B' in config else 10.0
    pool_size = int(config['pool_size']) if 'pool_size' in config else 50

    to_restore = False
    base_lr = float(config['base_lr']) if 'base_lr' in config else 0.0002
    max_step = int(config['max_step']) if 'max_step' in config else 100
    do_flipping = bool(config['do_flipping'])

    # 设置pch的init
    pch_model = PCH(pool_size, lambda_a, lambda_b, log_dir,
                              to_restore, base_lr, max_step,
                              checkpoint_dir, do_flipping, skip, kfold)

    pch_model.train()


if __name__ == '__main__':
    main(log_dir='./output', config_filename='./configs/exp_01.json', checkpoint_dir='', skip=True, kfold=1)

['\\data\\ct\\ct_train\\image\\ct_train_1001_image.nii\\z\\43.png', '\\data\\ct\\ct_train\\image\\ct_train_1001_image.nii\\z\\44.png', '\\data\\ct\\ct_train\\image\\ct_train_1001_image.nii\\z\\45.png', '\\data\\ct\\ct_train\\image\\ct_train_1001_image.nii\\z\\46.png', '\\data\\ct\\ct_train\\image\\ct_train_1001_image.nii\\z\\47.pn']
(256, 256, 1)
(256, 256, 3)
(256, 256, 3)
(256, 256, 4)
(256, 256, 4)


AttributeError: module 'tensorflow._api.v2.train' has no attribute 'shuffle_batch'